# PROG8245-25S-Sec1 Lab 2
Erica Holden, 5490685

In [28]:
%pip install pandas

import pandas as pd
from datetime import datetime


Note: you may need to restart the kernel to use updated packages.


## 1 Hello, Data!

In [2]:
df = pd.read_csv('./data/sales_records.csv')
print(df.head(3))

         date  customer_id     product   price  quantity coupon_code  \
0  2023-12-24         1087  Smartphone  699.99         1         NaN   
1  2023-04-12         1055      Camera  499.99         3  DISCOUNT20   
2  2023-11-03         1045      Tablet  399.99         2         NaN   

  shipping_city  
0      San Jose  
1  Philadelphia  
2  Philadelphia  


## 2 Pick the Right Container

I chose to represent the data as a class so I could use the key names I choose. A dict may have also been an option but using a class allows for certain OOP methods to be used with the code.

## 3 Transaction Class and OO data structure

In [ ]:
class Transaction:
    ''' This is a Transaction class which will hold data about each transaction in the sales set'''
    def __init__(self, date, customer_id, product, price, quantity, coupon_code, shipping_city):
        self.order_date = date
        self.customer_id = customer_id
        self.product = product
        self.price = price
        self.quantity = quantity
        self.coupon_code = coupon_code
        self.shipping_city = shipping_city


## 4 Bulk Loader

In [71]:
class Promo:
    ''' This is a promo class which will hold details about each promo'''
    def __init__(self, code, amount):
        self.code = code
        self.discount_amount = amount

class PromoLoader:
    ''' This is a promo loader class which will load promo codes and values based on a file path'''
    # Load the promos from a file given a specific path
    def loadPromos(self, file_path):
        promos = []
        df = pd.read_csv(file_path)
        for index, row in df.iterrows():
            promo = Promo(
                code=row['code'],
                amount=row['amount']
            )
            promos.append(promo)
        self.promo_list = promos
    
    # Simply return the promos available as a list
    def getPromos(self):
        return self.promo_list
    
class Loader:
    ''' This is a class made for loading and processing data given a file path'''
    # Load the transactions using the Transaction class, given a file path
    def loadTransactions(self, file_path):
        transactions = []
        df = pd.read_csv(file_path)
        for index, row in df.iterrows():
            transaction = Transaction(
                date=row['date'],
                customer_id=row['customer_id'],
                product=row['product'],
                price=row['price'],
                quantity=row['quantity'],
                coupon_code=row['coupon_code'],
                shipping_city=row['shipping_city']
            )
            transactions.append(transaction)
        self.tx_list = transactions

    # Simply return the list of the transactions in this object
    def getList(self):
        return self.tx_list
    
    # Get the minimum price for the transactions in this object
    def getMinPrice(self):
        min_price = self.tx_list[0].price

        for tx in self.tx_list:
            if tx.price < min_price:
                min_price = tx.price       
        return min_price
    
    # Get the maximum price for the transactions in this object
    def getMaxPrice(self):
        max_price = self.tx_list[0].price

        for tx in self.tx_list:
            if tx.price > max_price:
                max_price = tx.price
        return max_price

    # Get the mean price for the transactions in this object
    def getMeanPrice(self):
        total = 0
        count = 0
        mean = 0
        for tx in self.tx_list:
            total += tx.price
            count += 1
        
        mean = total / count
        return mean
    
    # Check if the given date is a valid date using pandas    
    def isValidDate(self, dt):
        try:
            pd.to_datetime(dt)
            return True
        except (ValueError, TypeError):
            return False

    # Clean the data and display the count of rows during the cleaning
    def clean(self):
        local_tx_list = self.tx_list.copy()
        print(f'Before cleaning, row count is: {len(local_tx_list)}')
        print(f'Cleaning rows with "LA" as the city instead of Los Angeles (could also transform later)')
        for tx in local_tx_list:
            if tx.shipping_city == 'LA':
                local_tx_list.remove(tx)
        print(f'After removing "LA" rows: {len(local_tx_list)}')
        print(f'Cleaning rows with invalid dates')
        for tx in local_tx_list:
            if (not self.isValidDate(tx.order_date)):
                local_tx_list.remove(tx)
        print(f'After removing rows with an invalid date: {len(local_tx_list)}')
        self.tx_list = local_tx_list
    
    # Transform the data to apply a discount amount to the row if there is a matching promo code, and calculate the sales
    # Also add an order ID to each order, since one was not provided in the dataset
    def transform(self):
        promoLoader = PromoLoader()
        promoLoader.loadPromos('./data/coupon_codes.csv')
        promosList = promoLoader.getPromos()

        for tx in self.tx_list:
            coupon_code = tx.coupon_code
            matches = [p for p in promosList if p.code == coupon_code]
            tx.discount = matches[0].discount_amount if matches else None
        
        for tx in self.tx_list:
            total_before_discount = tx.price * tx.quantity
            # Convert the discount to a floating point percentage
            discount_amount = (tx.discount / 100) * total_before_discount if tx.discount else 0
            tx.sales = total_before_discount - discount_amount

        order_index = 0
        for tx in self.tx_list:
            tx.order_id = order_index
            order_index += 1



    # Add a feature to show days since the purchase of the item(s)
    def addDaysSincePurchase(self):
        today = datetime.today()
        for tx in self.tx_list:
            purchase_date = datetime.strptime(tx.order_date, '%Y-%m-%d')
            tx.days_since_purchase = (today - purchase_date).days


## 5 Quick Profiling

In [72]:
loader = Loader();
loader.loadTransactions('./data/sales_records.csv')
tx_list = loader.getList()

print(f'Minimum price: {loader.getMinPrice()}')
print(f'Mean price: {loader.getMeanPrice()}')
print(f'Maximum price: {loader.getMaxPrice()}')

Minimum price: 49.99
Mean price: 362.6499999999991
Maximum price: 999.99


## 6 Spot the Grime

There are rows that are missing a city.
There are rows that use different names for the same city (LA and Los Angeles).
There is a row with a date formatted incorrectly.


## 7 Cleaning Rules

In [73]:
loader.clean()

Before cleaning, row count is: 500
Cleaning rows with "LA" as the city instead of Los Angeles (could also transform later)
After removing "LA" rows: 498
Cleaning rows with invalid dates
After removing rows with an invalid date: 497


## 8 Transformations

In [74]:
loader.transform()

## 9 Feature Engineering

In [75]:
loader.addDaysSincePurchase()

## 10 Mini-Aggregation

In [ ]:

cleaned_and_transformed_data = pd.DataFrame([vars(t) for t in loader.getList()])
sales_by_city = cleaned_and_transformed_data.groupby('shipping_city')['sales'].sum()
print(f'Total Sales by City: \n{sales_by_city}')
 
# for ld in loader.getList():
#    print(vars(ld))

shipping_city
Chicago         61454.5265
Dallas          52924.8890
Houston         56045.4290
Los Angeles     28866.3805
New York        60627.0550
Philadelphia    49652.1520
Phoenix         37464.3075
San Antonio     53429.9975
San Diego       49081.4840
San Jose        55963.4675
Name: sales, dtype: float64


## 11 Serialization Checkpoint	Save cleaned data to JSON and Parquet

## 12 Soft Interview Reflection	Markdown: < 120 words explaining how OOP helped